# Adding paths and files

In [109]:
import os
import glob
import polars as pl

# Specify the directory where your CSV files are located
directory = r'D:\github\localt20\data\t20s_csv2'

# Use glob to find all CSV files in the specified directory
info_files = glob.glob(os.path.join(directory, '*_info.csv'))
all_files = glob.glob(os.path.join(directory,'*.csv'))
delivery_files = [file for file in all_files if '_info' not in file]

matches=[]
deliveries=[]
# Print the list of CSV files
for info_file in info_files:
    matches.append(info_file.split('\\')[-1])
for delivery in delivery_files:
    if '_info' not in delivery:
        deliveries.append(delivery.split('\\')[-1])

In [110]:
import pandas as pd

df = pd.read_csv(info_files[0], header=None, names=['type', 'heading', 'subkey', 'players','player_id'], skipinitialspace=True).drop('type', axis=1)
df.head(10)

,heading,subkey,players,player_id
0,2.1.0,NaN,NaN,NaN
1,balls_per_over,6,NaN,NaN
2,team,Australia,NaN,NaN
3,team,Sri Lanka,NaN,NaN
4,gender,male,NaN,NaN
5,season,2016/17,NaN,NaN
6,date,2017/02/17,NaN,NaN
7,event,Sri Lanka in Australia T20I Series,NaN,NaN
8,match_number,1,NaN,NaN
9,venue,Melbourne Cricket Ground,NaN,NaN


In [111]:
df['subkey'][5]

'2016/17'

In [112]:
match_id = pd.to_numeric(info_files[0].split('\\')[-1].split('_')[0])
match_id

1001349

In [113]:
# Filter dataframes based on the heading
players_df = df[df['heading'] == "player"].drop(['heading','player_id'], axis=1)
registry_df = df[df['heading'] == "registry"].drop('heading', axis=1)

# Join on the 'players' column with 'player_id' from the registry dataframe
merged_df = players_df.merge(registry_df[['players', 'player_id']], on='players', how='inner')

# Display the merged dataframe
merged_df.rename(columns={'players':'player','subkey':'country'}, inplace=True)
merged_df['match_id'] = match_id
merged_df

,country,player,player_id,match_id
0,Australia,AJ Finch,b8d490fd,1001349
1,Australia,M Klinger,b970a03f,1001349
2,Australia,TM Head,12b610c2,1001349
3,Australia,MC Henriques,32198ae0,1001349
4,Australia,AJ Turner,ff1e12a0,1001349
5,Australia,JP Faulkner,808f425a,1001349
6,Australia,TD Paine,5748e866,1001349
7,Australia,PJ Cummins,ded9240e,1001349
8,Australia,A Zampa,14f96089,1001349
9,Australia,B Stanlake,6834d1f2,1001349


In [114]:
dataframes = pd.DataFrame(columns=['country', 'player','player_id','season','match_id'])
injured_matches = []

for info_file in info_files:
    match_id = pd.to_numeric(info_file.split('\\')[-1].split('_')[0])
    try:
        df = pd.read_csv(info_file, header=None, names=['type', 'heading', 'subkey', 'players','player_id'], skipinitialspace=True).drop('type', axis=1)
        players_df = df[df['heading'] == "player"].drop(['heading','player_id'], axis=1)
        registry_df = df[df['heading'] == "registry"].drop('heading', axis=1)
        merged_df = players_df.merge(registry_df[['players', 'player_id']], on='players', how='inner')
        merged_df.rename(columns={'players':'player','subkey':'country'}, inplace=True)
        season = df['subkey'][5] 
        merged_df['match_id'] = match_id
        merged_df['season'] = season
        if(len(merged_df)!=22):
            raise Exception('Injured Match')
        dataframes = pd.concat([dataframes, merged_df])
    except:
        injured_matches.append(match_id)
print(injured_matches)

[1173070, 1223952, 1251954, 1262758, 1262760, 1263164, 1263166, 1263167, 1267682, 1268757, 1270834, 1270835, 1273136, 1273138, 1273144, 1274596, 1274597, 1286970, 1289274, 1298163, 1362817, 1382164, 1388204, 1393329, 1411261, 1416079, 1425126, 1425660, 1434291, 1443786, 1443789, 1444961, 1449012, 222678]


In [120]:
dataframes

,country,player,player_id,match_id,season
0,Australia,AJ Finch,b8d490fd,1001349,2016/17
1,Australia,M Klinger,b970a03f,1001349,2016/17
2,Australia,TM Head,12b610c2,1001349,2016/17
3,Australia,MC Henriques,32198ae0,1001349,2016/17
4,Australia,AJ Turner,ff1e12a0,1001349,2016/17
...,...,...,...,...,...
17,Sri Lanka,NLTC Perera,0f12f9df,995469,2016
18,Sri Lanka,SS Pathirana,753c95b9,995469,2016
19,Sri Lanka,S Prasanna,f78e7113,995469,2016
20,Sri Lanka,SMSM Senanayake,4c4fa80b,995469,2016


In [121]:
len(dataframes)/22,len(injured_matches)

(3764.0, 34)

In [122]:
dataframes.to_csv("../../processedData/Matchplayers.csv",index=False)

# Individual player's data

In [118]:
players = pl.from_pandas(dataframes).drop('match_id').select('player','country','player_id').unique()
players

player,country,player_id
str,str,str
"""PS Athavale""","""Oman""","""c2f12310"""
"""HV Mandhyan""","""Hungary""","""11f56b14"""
"""L Kooagile""","""Botswana""","""88c742fb"""
"""T Maruma""","""Zimbabwe""","""f55c526f"""
"""AA Obanda""","""Kenya""","""7081491e"""
…,…,…
"""DR Patankar""","""Malta""","""e661915c"""
"""B Kumar""","""India""","""2e81a32d"""
"""LMM Tahuhu""","""New Zealand""","""60471339"""


In [119]:
players.write_csv("../../processedData/Players.csv")